<a href="https://colab.research.google.com/github/theotaburet/Keeping_with_Python/blob/main/YOHO-Urban-SED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
import glob

In [15]:
pip install soundata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 34.8 MB/s eta 0:00:00
  Created wheel for jams: filename=jams-0.3.4-py3-none-any.whl size=64901 sha256=0228205cf27944944c22355f717ed14344a7fa07cfb9e791fba5e9f482964e7d
  Stored in director

In [ ]:
import soundata

dataset = soundata.initialize('urbansound8k')
dataset.download()  # download the dataset
dataset.validate()  # validate that all the expected files are there

 63%|██████▎   | 3.53G/5.61G [10:48<02:50, 13.1MB/s]

In [12]:
zip_name = "train-data.zip"
with ZipFile(zip_name, 'r') as zip:
  zip.extractall()

BadZipFile: File is not a zip file

In [13]:
zip_name = "/content/val-data.zip"
with ZipFile(zip_name, 'r') as zip:
  zip.extractall()

BadZipFile: File is not a zip file

In [ ]:
import shutil

In [ ]:
# g = glob.glob("/content/drive/MyDrive/TUT-sound-events-2017/Development/*.zip")

# for gg in g:
zip_name = "/content/Urban-SED-V2.tar.gz"

shutil.unpack_archive(zip_name, "/content/Urban-SED/")

# Annotations

In [ ]:
import csv

In [ ]:
def read_annotation(filename):
    events = []
    with open(filename, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in spamreader:
            events.append(row)
    return events

In [ ]:
events = read_annotation("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/train/soundscape_train_bimodal0.txt")

In [ ]:
events

[['3.1254220000000004', '3.891403', 'jackhammer'],
 ['4.272823000000002', '6.293218000000001', 'dog_bark'],
 ['4.644566', '7.936222000000001', 'drilling']]

In [ ]:
import soundfile as sf

In [ ]:
from subprocess import Popen, PIPE
from os.path import dirname
import os

# Split into folds

In [ ]:
train_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/train/*.wav")
val_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/*.wav")

In [ ]:
train_ann = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/train/*.txt")
val_ann = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/annotations/validate/*.txt")

In [ ]:
len(train_ann)

6000

In [ ]:
import soundfile as sf

In [ ]:
a, sr = sf.read(train_files[0])


In [ ]:
int(a.shape[0]/10.0)

44100

In [ ]:
len(train_files)

6000

In [ ]:
import math
import numpy as np

In [ ]:
def construct_examples(audio_path, win_len = 2.56, hop_len = 1.0, sr = 44100.0):

  win_len_t = win_len
  hop_len_t = hop_len

  win_len = int(sr*win_len)
  hop_len = int(sr*hop_len)

  a, sr = sf.read(audio_path)

  if a.shape[0] < win_len:
    a_padded = np.zeros((win_len, ))
    a_padded[0:a.shape[0]] = a

  else:
    no_of_hops = math.ceil((a.shape[0] - win_len) / hop_len)
    a_padded = np.zeros((int(win_len + hop_len*no_of_hops), ))
    a_padded[0:a.shape[0]] = a

  a_ex = [a_padded[i - win_len : i] for i in range(win_len, a_padded.shape[0]+1, hop_len)]
  win_ranges = [((i - win_len)/sr, i/sr) for i in range(win_len, a_padded.shape[0]+1, hop_len)]

  return a_ex, win_ranges

In [ ]:
a, w = construct_examples(train_files[0])

In [ ]:
a

[array([-0.00036621, -0.00128174, -0.00164795, ..., -0.02493286,
        -0.0244751 , -0.02774048]),
 array([-0.00219727, -0.0015564 , -0.00128174, ..., -0.00537109,
        -0.00552368, -0.00616455]),
 array([-0.01119995, -0.01153564, -0.01223755, ..., -0.00671387,
        -0.00698853, -0.00765991]),
 array([-0.01794434, -0.01806641, -0.01925659, ..., -0.01611328,
        -0.01577759, -0.01550293]),
 array([-0.00683594, -0.00646973, -0.00662231, ..., -0.00076294,
        -0.00018311, -0.00057983]),
 array([ 0.00292969,  0.00222778,  0.00289917, ...,  0.00231934,
        -0.00247192, -0.0085144 ]),
 array([0.0043335 , 0.00466919, 0.00402832, ..., 0.01187134, 0.02832031,
        0.03540039]),
 array([-0.01068115, -0.00595093, -0.00115967, ...,  0.06747437,
         0.05987549,  0.0489502 ]),
 array([-0.01016235, -0.01049805, -0.012146  , ...,  0.        ,
         0.        ,  0.        ])]

In [ ]:
w

[(0.0, 2.56),
 (1.0, 3.56),
 (2.0, 4.56),
 (3.0, 5.56),
 (4.0, 6.56),
 (5.0, 7.56),
 (6.0, 8.56),
 (7.0, 9.56),
 (8.0, 10.56)]

In [ ]:
def construct_labels(annotation_path, win_start, win_end, win_len):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]

  curr_ann = []

  for a in ann:
    if a[1] > win_start and a[0] <= win_end:
    # if a[0] >= win_start and a[0] < win_end:
      curr_start = max(a[0] - win_start, 0.0)
      curr_end = min(a[1] - win_start, win_len)
      curr_ann.append([curr_start, curr_end, a[2]])

  class_set = set([c[2] for c in curr_ann])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in curr_ann:
    class_wise_events[c[2]].append(c)

  max_event_silence = 0.0
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events

In [ ]:
class_dict = {'air_conditioner': 0,
              'car_horn': 1,
              'children_playing': 2,
              'dog_bark': 3,
              'drilling': 4,
              'engine_idling': 5,
              'gun_shot': 6,
              'jackhammer': 7,
              'siren': 8,
              'street_music': 9}

#air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren and street_music

In [ ]:
def to_seg_by_class(events, class_dict, hop_len = 441, n_frames = 1001, sr=44100):
  # events = smoothe_events(events)
  labels = np.zeros((n_frames, 10), dtype=np.float32)

  for e in events:
    t1 = float(e[0])
    t1 = int(t1 / hop_len * sr)
    t2 = float(e[1])
    t2 = int(t2 / hop_len * sr)

    labels[t1:t2, class_dict[e[2]]] = 1

  return labels

In [ ]:
def get_universal_labels(events, class_dict, ex_length = 10.0, no_of_div = 32):
  win_length = ex_length/no_of_div
  labels = np.zeros((no_of_div, len(class_dict.keys()) * 3))

  for e in events:

    start_time = float(e[0])
    stop_time = float(e[1])

    start_bin = int(start_time // win_length)
    stop_bin = int(stop_time // win_length)

    start_time_2 = start_time - start_bin * win_length
    stop_time_2 = stop_time - stop_bin * win_length

    n_bins = stop_bin - start_bin

    if n_bins == 0:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, stop_time_2]

    elif n_bins == 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

    elif n_bins > 1:
      labels[start_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, start_time_2, win_length]

      for i in range(1, n_bins):
        labels[start_bin + i, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, win_length]

      if stop_time_2 > 0.0:
        labels[stop_bin, class_dict[e[2]] * 3:class_dict[e[2]] * 3 + 3] = [1, 0.0, stop_time_2]

  # labels[:, [1, 2, 4, 5]] /= win_length

  for i in range(len(labels)):
    for j in range(len(labels[i])):
      if j % 3 != 0:
        labels[i][j] /= win_length

  return labels

In [ ]:
import shutil

In [ ]:
os.mkdir("/content/content/")

In [ ]:
# shutil.rmtree("/content/content/train-data")
os.mkdir("/content/content/train-data")

In [ ]:
mel_files = glob.glob('/content/train-data/*')
print(len(mel_files))


0


In [ ]:
"""
Store Urban-SED melspectrograms in drive to save time for later runs
"""
mel_files = glob.glob('/content/val-data/*')
print(mel_files)

zip_file_name = "/content/drive/MyDrive/Urban-SED/val-data.zip"

with ZipFile(zip_file_name, 'w') as my_zip:
  for f in mel_files:
    my_zip.write(f)

['/content/val-data/label-633.npy', '/content/val-data/label-1789.npy', '/content/val-data/ex-18.npy', '/content/val-data/ex-320.npy', '/content/val-data/ex-932.npy', '/content/val-data/label-269.npy', '/content/val-data/ex-1399.npy', '/content/val-data/ex-951.npy', '/content/val-data/ex-27.npy', '/content/val-data/label-376.npy', '/content/val-data/ex-758.npy', '/content/val-data/ex-109.npy', '/content/val-data/ex-13.npy', '/content/val-data/ex-1480.npy', '/content/val-data/ex-1336.npy', '/content/val-data/label-1968.npy', '/content/val-data/label-917.npy', '/content/val-data/ex-1887.npy', '/content/val-data/label-0.npy', '/content/val-data/ex-1908.npy', '/content/val-data/ex-1418.npy', '/content/val-data/label-228.npy', '/content/val-data/label-48.npy', '/content/val-data/ex-1542.npy', '/content/val-data/ex-1459.npy', '/content/val-data/label-1698.npy', '/content/val-data/label-1954.npy', '/content/val-data/label-865.npy', '/content/val-data/label-1557.npy', '/content/val-data/ex-570

In [ ]:
import librosa

In [ ]:
def get_log_melspectrogram(audio, sr = 44100, hop_length = 441, win_length = 1764, n_fft = 2048, n_mels = 40, fmin = 0, fmax = 22050):
    """Return the log-scaled Mel bands of an audio signal."""
    audio_2 = librosa.util.normalize(audio)
    bands = librosa.feature.melspectrogram(
        y=audio_2, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels)
    return librosa.core.power_to_db(bands)

In [ ]:
"""
Construct train set
"""

win_len = 10.0
hop_len = 10.0
a_ex_train = []
a_labels_train = []

# win_size = 10.0
# win_start = 16.0
# win_end = win_start + win_size

for i, audio in enumerate(train_files):
  a, win_ranges = construct_examples(audio,win_len=win_len, hop_len=hop_len)
  M = get_log_melspectrogram(a[0]).T
  np.save("/content/content/train-data/ex-" + str(i) + ".npy", M)

  # a_ex_train += a

  for w in win_ranges:
    labels_t = construct_labels(audio.replace(".wav", ".txt").replace("audio", "annotations"), w[0], w[1], win_len=win_len)
    # ll = get_universal_labels(labels_t, class_dict, ex_length=win_len, no_of_div = 32)
    ll = to_seg_by_class(labels_t, class_dict)
    # a_labels_train.append(ll)
    np.save("/content/content/train-data/label-" + str(i) + ".npy", ll)

    # a_labels_train.append(to_seg_by_class(labels_t, class_dict))

In [ ]:
# a, sr = sf.read(audio_files_mono[0])
a, sr = sf.read(train_files[0])
M = get_log_melspectrogram(a)

In [ ]:
ff = glob.glob("/content/train-data/*")
print(len(ff))

0


In [ ]:
M.shape

(40, 1001)

In [ ]:
# shutil.rmtree("/content/content/val-data")
os.mkdir("/content/content/val-data")

In [ ]:
len(val_files)

2000

In [ ]:
"""
Construct val set
"""

win_len = 10.0
hop_len = 10.0
a_ex_val = []
a_labels_val = []

# win_size = 10.0
# win_start = 16.0
# win_end = win_start + win_size

for i, audio in enumerate(val_files):
  a, win_ranges = construct_examples(audio,win_len=win_len, hop_len=hop_len)
  M = get_log_melspectrogram(a[0]).T
  np.save("/content/content/val-data/ex-" + str(i) + ".npy", M)
  # a_ex_val += a

  for w in win_ranges:
    labels_t = construct_labels(audio.replace(".wav", ".txt").replace("audio", "annotations"), w[0], w[1], win_len=win_len)
    # ll = get_universal_labels(labels_t, class_dict, ex_length=win_len, no_of_div = 32)
    ll = to_seg_by_class(labels_t, class_dict)
    # a_labels_val.append(ll)
    np.save("/content/content/val-data/label-" + str(i) + ".npy", ll)

    # a_labels_train.append(to_seg_by_class(labels_t, class_dict))

In [ ]:
win_ranges

[(0.0, 10.0)]

In [ ]:
import re

def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [ ]:
import glob
import random
"""
Load the individual numpy arrays into partition
"""
data = glob.glob("/content/content/train-data/ex-*.npy") # + glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True)
#data = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-[0-9]*.npy", recursive=True)
sort_nicely(data)

labels = glob.glob("/content/content/train-data/label-*.npy") #+ glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
#labels = glob.glob("/content/train data/MuSpeak/content/Mel Files/**/mel-id-label-[0-9]*.npy", recursive=True)
sort_nicely(labels)

train_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(train_examples)
#print(train_examples[0])

# m = len(train_examples)
# m_validation = 1024
# m_test = 512
# m_train = 40960

# partition = {}
# partition['train'] = train_examples[0:m_train]
# partition['validation'] = examples[m_train:m_train + m_validation]
# partition['test'] = examples[m_train + m_validation:m]

In [ ]:
"""
Creating the train partition.
"""
partition = {}
partition['train'] = train_examples

random.shuffle(partition['train'])

In [ ]:
"""
This loads data for the validation set.
"""
import glob
import random

data = glob.glob("/content/content/val-data/ex-*.npy")
sort_nicely(data)

labels = glob.glob("/content/content/val-data/label-*.npy")
sort_nicely(labels)

validation_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(validation_examples)
print(validation_examples[0])

# m = len(test_examples)
# m_validation = 1024
# m_test = 512
# m_train = m - m_validation - m_test

partition['validation'] = validation_examples

('/content/content/val-data/ex-1211.npy', '/content/content/val-data/label-1211.npy')


In [ ]:
len(partition['train'])

6000

In [ ]:
!git clone https://github.com/DemisEom/SpecAugment.git
!pip install /content/SpecAugment/
!pip install tensorflow-addons

Cloning into 'SpecAugment'...
remote: Enumerating objects: 271, done.
remote: Total 271 (delta 0), reused 0 (delta 0), pack-reused 271
Receiving objects: 100% (271/271), 429.45 KiB | 13.85 MiB/s, done.
Resolving deltas: 100% (150/150), done.
Processing ./SpecAugment
  Created wheel for SpecAugment: filename=SpecAugment-1.2.3-cp37-none-any.whl size=16700 sha256=7a40f61d7ab1996e2ef1073908319fa584aca23cbc2d0599d5beb3df481ca96b
  Stored in directory: /tmp/pip-ephem-wheel-cache-wv9t7_yz/wheels/fc/d9/61/5583932096e24004288b272f47e80f2d9e0325234944010447
Successfully built SpecAugment
     |████████████████████████████████| 686kB 7.1MB/s 


In [ ]:
from SpecAugment import spec_augment_tensorflow

In [ ]:
!pip install tensorflow-io

     |████████████████████████████████| 22.7MB 1.4MB/s 
     |████████████████████████████████| 2.3MB 50.8MB/s 


In [ ]:
import tensorflow_io as tfio

In [ ]:
import tensorflow as tf
import keras

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_examples, batch_size=128, epoch_size = 16384, dim=(1, ),
                 n_classes=2, shuffle=True):
        'Initialization'
        print("Constructor called!!!")
        self.dim = dim
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        self.list_examples = list_examples
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        #print("The self.list_examples is {}".format(self.list_examples))
        return int(np.floor(len(self.list_examples) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_examples[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
      self.indexes = np.arange(len(self.list_examples))
      if self.shuffle == True:
          np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # # Initialization
        X = np.empty([self.batch_size, 1001, 40, 1], dtype=np.float64)
        y = np.empty([self.batch_size, 1001, 10], dtype=np.float64)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
          # Store sample

          xx = np.load(ID[0])

          # v = xx.shape[0]
          # tau = xx.shape[1]

          # xx = tf.expand_dims(xx, axis=0)
          # xx = tf.expand_dims(xx, axis=-1)

          # # warped_mel_spectrogram = sparse_warp(mel_spectrogram)

          # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(xx, v=v, frequency_masking_para=8, frequency_mask_num=1)

          # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)



          # xx = tf.expand_dims(xx, axis=0)
          # xx = tf.expand_dims(xx, axis=-1)



          # freq_mask = tfio.audio.freq_mask(xx, param=8)

          # time_mask = tfio.audio.time_mask(freq_mask, param=25)

          # xx = tf.squeeze(warped_frequency_time_sepctrogram)
          X[i, :, :, 0] = xx

          # Store class
          yy = np.load(ID[1])
          # yy2 = yy[:, [1, 2, 4, 5]]
          y[i, :, :] = yy



        # tau = X.shape[1]
        # v = X.shape[2]

        # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
        # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=3)

        # X = warped_frequency_time_sepctrogram



        return X, y

In [ ]:
# Parametersa
params = {'dim': (1, ),
          'batch_size': 32,
          'epoch_size': 0,
          'n_classes': 2,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

Constructor called!!!
Constructor called!!!


In [ ]:
# g = glob.glob("/content/train-data/*.npy")
# print(g)
nn = np.load("/content/val-data/ex-0.npy")

FileNotFoundError: ignored

In [ ]:
nn.shape

NameError: ignored

# Splitting into folds

# Audio feature extraction

In [ ]:
a_labels_train[0].shape

(1001, 6)

In [ ]:
len(a_ex_train)

529

In [ ]:
a_ex_train[0].shape

(160000,)

In [ ]:
import librosa

In [ ]:
# def get_log_melspectrogram(audio, sr = 16000, hop_length = 160, win_length = 400, n_fft = 512, n_mels = 64, fmin = 125, fmax = 7500):
#     """Return the log-scaled Mel bands of an audio signal."""
#     bands = librosa.feature.melspectrogram(
#         y=audio, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels, fmin=fmin, fmax=fmax, dtype=np.float32)
#     return librosa.core.power_to_db(bands)

In [ ]:
M = get_log_melspectrogram(a_ex_train[0]).T

In [ ]:
M.shape

(1001, 64)

In [ ]:
X_train = np.zeros((len(a_ex_train), M.shape[0], M.shape[1]))
Y_train = np.zeros((len(a_ex_train), a_labels_train[0].shape[0], a_labels_train[0].shape[1]))

In [ ]:
X_val = np.zeros((len(a_ex_val), M.shape[0], M.shape[1]))
Y_val = np.zeros((len(a_ex_val), a_labels_val[0].shape[0], a_labels_val[0].shape[1]))

In [ ]:
for i in range(len(a_ex_train)):
  X_train[i, :, :] = get_log_melspectrogram(a_ex_train[i]).T
  Y_train[i, :, :] = a_labels_train[i]

In [ ]:
for i in range(len(a_ex_val)):
  X_val[i, :, :] = get_log_melspectrogram(a_ex_val[i]).T
  Y_val[i, :, :] = a_labels_val[i]

In [ ]:
X_train[-1]

array([[ -32.14020031,  -40.35559271,  -39.6755155 , ...,  -65.58998433,
         -59.99374257,  -65.58119134],
       [ -32.06349092,  -34.69988957,  -36.04470892, ...,  -64.54238997,
         -65.8907304 ,  -71.46275124],
       [ -30.46371327,  -31.56644259,  -34.6040171 , ...,  -66.02762789,
         -65.26727856,  -70.52418497],
       ...,
       [-100.        , -100.        , -100.        , ..., -100.        ,
        -100.        , -100.        ],
       [-100.        , -100.        , -100.        , ..., -100.        ,
        -100.        , -100.        ],
       [-100.        , -100.        , -100.        , ..., -100.        ,
        -100.        , -100.        ]])

# Define the network

In [ ]:
def my_loss_fn(y_true, y_pred):
  weight = tf.constant([1.0])

  squared_difference = tf.square(y_true - y_pred)

  # squared_difference[:, 1] =  squared_difference[:, 1] * y_true[:, 0]

  # squared_difference[:, 3] =  squared_difference[:, 3] * y_true[:, 2]

  ss_True = squared_difference[:, :, 0] * 0 + 1

  ss_0 = y_true[:, :, 0]
  ss_1 = y_true[:, :, 3]
  ss_2 = y_true[:, :, 6]
  ss_3 = y_true[:, :, 9]
  ss_4 = y_true[:, :, 12]
  ss_5 = y_true[:, :, 15]
  ss_6 = y_true[:, :, 18]
  ss_7 = y_true[:, :, 21]
  ss_8 = y_true[:, :, 24]
  ss_9 = y_true[:, :, 27]

  sss = tf.stack((ss_True, ss_0, ss_0,
                  ss_True, ss_1, ss_1,
                  ss_True, ss_2, ss_2,
                  ss_True, ss_3, ss_3,
                  ss_True, ss_4, ss_4,
                  ss_True, ss_5, ss_5,
                  ss_True, ss_5, ss_6,
                  ss_True, ss_5, ss_7,
                  ss_True, ss_5, ss_8,
                  ss_True, ss_5, ss_9), axis = 2)

  squared_difference =  tf.multiply(squared_difference, sss)

  # mask_1 = tf.cast(tf.equal(ss1, 0), dtype = np.float32)
  # mask_1 = mask_1 * weight

  # mask_2 = tf.cast(tf.equal(ss1, 1), dtype = np.float32)

  # mask_3 = tf.cast(tf.equal(ss4, 0), dtype = np.float32)
  # mask_3 = mask_3 * weight

  # mask_4 = tf.cast(tf.equal(ss4, 1), dtype = np.float32)

  # mask_s = mask_1 + mask_2

  # mask_m = mask_3 + mask_4

  # mmm = tf.stack((mask_s, ss0, ss0, mask_m, ss3, ss3), axis = 2)

  # squared_difference =  tf.multiply(squared_difference, mmm)

  return tf.reduce_sum(squared_difference, axis=[-1, -2])  # Note the `axis=-1`

In [ ]:
def binary_acc(y_true, y_pred):
  threshold = tf.constant([0.5])

  binary_true = tf.stack((y_true[:, :, 0], y_true[:, :, 3], y_true[:, :, 6], y_true[:, :, 9], y_true[:, :, 12],
                          y_true[:, :, 15], y_true[:, :, 18], y_true[:, :, 21], y_true[:, :, 24], y_true[:, :, 27]), axis=1)
  binary_pred = tf.stack((y_pred[:, :, 0], y_pred[:, :, 3], y_pred[:, :, 6], y_pred[:, :, 9], y_pred[:, :, 12]
                          , y_pred[:, :, 15], y_pred[:, :, 18], y_pred[:, :, 21], y_pred[:, :, 24], y_pred[:, :, 27]), axis=1)

  binary_true = tf.greater_equal(binary_true, threshold)
  binary_pred = tf.greater_equal(binary_pred, threshold)

  # binary_pred = tf.greater(y_pred[:, [0, 2]], threshold)

  # acc = tf.square(y_true - y_pred)
  acc = tf.cast((binary_true == binary_pred), tf.float32)

  return tf.reduce_mean(acc, axis=[-1, -2])  # Note the `axis=-1`

In [ ]:
import tensorflow as tf

In [ ]:
# This optimises val loss for Wave-U-Net YOHO
# Back to Val Binary acc.

import os
class MyCustomCallback_3(tf.keras.callbacks.Callback):
  def __init__(self, model_dir, patience=0):
    super(MyCustomCallback_3, self).__init__()
    self.patience = patience
    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None
    self.model_best_path = os.path.join(model_dir, 'model-best.h5')
    self.model_last_path = os.path.join(model_dir, 'model-last-epoch.h5')
    self.custom_params = {"best_loss":np.inf, "last_epoch":0}

    self.custom_params_path = os.path.join(model_dir, 'custom_params.pickle')
    if os.path.isfile(self.custom_params_path):
      with open(self.custom_params_path, 'rb') as f:
        self.custom_params = pickle.load(f)
      # best_model = tf.keras.models.load_model(self.model_best_path, custom_objects={'my_loss_fn':my_loss_fn,
      #             'binary_acc':binary_acc, 'TCN':TCN(), 'SpeechF1':SpeechF1(), 'MusicF1':MusicF1(),
      #             'OnAcc':OnAcc(), 'OnOffAcc':OnOffAcc(),'AudioClipLayer':AudioClipLayer(),
      #             'InterpolationLayer':InterpolationLayer(), 'CropLayer':CropLayer(),
      #             'IndependentOutputLayer':IndependentOutputLayer(), 'DiffOutputLayer':DiffOutputLayer()})
      # best_model = tf.keras.models.clone_model(self.model)
      # best_model.load_weights(self.model_best_path)
      # self.best_weights = best_model.get_weights()


  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best F1 as 0.0.
    self.is_impatient = False

  def on_train_end(self, logs=None):
    if not self.is_impatient:
      print("Restoring model weights from the end of the best epoch.")
      self.model.set_weights(self.best_weights)
      # temp_model_path = self.model_path.replace(".h5", "_temp.h5")
      #os.remove(temp_model_path)

  def on_epoch_end(self, epoch, logs=None):
    current_val_loss = logs.get("val_loss")
    self.model.save_weights(self.model_last_path)
    self.custom_params["last_epoch"] = self.custom_params["last_epoch"] + 1

    if current_val_loss < self.custom_params['best_loss']:
      self.custom_params['best_loss'] = current_val_loss
      self.wait = 0
      self.best_weights = self.model.get_weights()
      self.model.save_weights(self.model_best_path)

    else:
        self.wait += 1
        if self.wait >= self.patience:
            self.stopped_epoch = epoch
            self.is_impatient = True
            self.model.stop_training = True
            print("Restoring model weights from the end of the best epoch.")
            self.model.set_weights(self.best_weights)
            #os.remove(temp_model_path)
    with open(self.custom_params_path, 'wb') as f:
      pickle.dump(self.custom_params, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
list(enumerate(val_files))

[(0,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_uniform637.wav'),
 (1,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_unimodal322.wav'),
 (2,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal8.wav'),
 (3,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal613.wav'),
 (4,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_uniform465.wav'),
 (5,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal411.wav'),
 (6,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal658.wav'),
 (7,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_uniform622.wav'),
 (8,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_uniform111.wav'),
 (9,
  '/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_unimodal431.wav'),
 (10,
  '/content/Urban-SED/UR

In [ ]:
len(val_files)

2000

In [ ]:
!pip install sed_eval

     |████████████████████████████████| 2.1 MB 18.6 MB/s 
  Created wheel for sed-eval: filename=sed_eval-0.2.1-py3-none-any.whl size=26125 sha256=2882082c4e929295e0353ee8630602ea8a36a008b605f6debfa29954145cedf5
  Stored in directory: /root/.cache/pip/wheels/96/19/00/a3ec89acfdbd2568185a1ae3020590e76ee73a47ac4ff83e84
  Created wheel for dcase-util: filename=dcase_util-0.2.18-py3-none-any.whl size=2147232 sha256=ad476ad54476009a23282dbbdd96b4acce987f2ec37e60913a3c260a0b3ed690
  Stored in directory: /root/.cache/pip/wheels/f2/75/d1/e4fc6f415d1100dc30c39c955aa2bfbd0d88c6138076e74793
Successfully built sed-eval dcase-util


In [ ]:
import sed_eval
import dcase_util

In [ ]:
import pickle

In [ ]:
# Creates mel spctrograms for training

win_length = 10.0
hop_size = 10.0
mss_ins = []
win_ranges_list = []
n_classes = 10


for ii, audio in enumerate(val_files):
  a, win_ranges = construct_examples(audio, win_len=win_length,hop_len=hop_size)

  mss_in = np.zeros((1, 1001, 40))

  # if len(a) > 1:
  #   print(audio)

  preds = np.zeros((len(a), 3, 30)) # I think this is an unused variable


  for i in range(1):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T
  mss_ins.append(mss_in)
  win_ranges_list.append(win_ranges)

In [ ]:
# from google.colab import files
# files.download('/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal528.wav')

In [ ]:
a, sr = sf.read("/content/Urban-SED/URBAN-SED_v2.0.0/audio/validate/soundscape_validate_bimodal528.wav")
a.shape[0]/sr

10.000022675736961

In [ ]:
len(mss_ins)

2000

In [ ]:
mss_ins_2 = np.concatenate(mss_ins, axis=0)

In [ ]:
mss_ins_2.shape

(2000, 1001, 40)

In [ ]:


# for i in range(len(mss_ins)):
#   mss_ins[i] = np.squeeze(mss_ins[i])

# mss_ins = np.array(mss_ins)


def mk_preds_YOHO_mel_2(model, mss_ins=mss_ins_2, no_of_div = 32, hop_size = 10.0, discard = 0.3, win_length = 10.0, max_event_silence = 0.3, sampling_rate = 44100):
  preds = model.predict(mss_ins_2)

  for ii, audio in enumerate(val_files):
    audio_file_path = audio

    events = []
    p = preds[ii, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)):
      for jjj in range(0, n_classes):
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges_list[ii][0][0]
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


    class_set = set([c[2] for c in events])
    class_wise_events = {}

    for c in list(class_set):
      class_wise_events[c] = []


    for c in events:
      class_wise_events[c[2]].append(c)


    all_events = []

    for k in list(class_wise_events.keys()):
      curr_events = class_wise_events[k]
      count = 0

      while count < len(curr_events) - 1:
        if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
          curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
          del curr_events[count + 1]
        else:
          count += 1

      all_events += curr_events

    for i in range(len(all_events)):
      all_events[i][0] = round(all_events[i][0], 3)
      all_events[i][1] = round(all_events[i][1], 3)

    all_events.sort(key=lambda x: x[0])



    see = all_events
    n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

    with open(n_label, 'w') as fp:
      fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))



import os

In [ ]:
rev_class_dict = ['air_conditioner', 'car_horn', 'children_playing',
               'dog_bark', 'drilling', 'engine_idling', 'gun_shot',
               'jackhammer','siren', 'street_music']

In [ ]:
def frames_to_time(f, sr = 44100.0, hop_size = 441):
  return f * hop_size / sr

def preds_to_se(p, win_start, audio_clip_length = 2.56):
  start_dicts = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
  stop_dicts = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


  start_speech = -100
  start_music = -100
  stop_speech = -100
  stop_music = -100

  audio_events = []

  n_frames = p.shape[0]

  for j in range(p.shape[1]):
    if p[0, j] >= 0.5:
      start_dicts[j] = 0

  for j in range(p.shape[1]):
    for i in range(n_frames - 1):
      if p[i, j] < 0.5 and p[i+1, j] >= 0.5:
        start_dicts[j] = i+1

      elif p[i, j] >= 0.5 and p[i + 1, j] < 0.5:
        stop_dicts[j] = i
        start_time = frames_to_time(start_dicts[j])
        stop_time = frames_to_time(stop_dicts[j])

        audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
        start_dicts[j] = -100
        stop_dicts[j] = -100

    if start_dicts[j] != -100:
      start_time = frames_to_time(start_dicts[j])
      stop_time = audio_clip_length
      audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
      start_dicts[j] = -100
      stop_dicts[j] = -100

  audio_events.sort(key = lambda x: x[0])
  return audio_events


def mk_preds_CRNN_mel_2(model, mss_ins=mss_ins_2, no_of_div = 9, hop_size = 10.0, discard = 0.3, win_length = 10.0, max_event_silence = 0.3, sampling_rate = 44100):
  preds = model.predict(mss_ins_2)


  for ii, audio in enumerate(val_files):
    audio_file_path = audio
    events = []
    p = preds[ii, :, :]
    events_curr = []

    events_curr = preds_to_se(p, win_start = win_ranges_list[ii][0][0], audio_clip_length=win_length)

    events += events_curr


    class_set = set([c[2] for c in events])
    class_wise_events = {}

    for c in list(class_set):
      class_wise_events[c] = []


    for c in events:
      class_wise_events[c[2]].append(c)


    all_events = []

    for k in list(class_wise_events.keys()):
      curr_events = class_wise_events[k]
      count = 0

      while count < len(curr_events) - 1:
        if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
          curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
          del curr_events[count + 1]
        else:
          count += 1

      all_events += curr_events

    for i in range(len(all_events)):
      all_events[i][0] = round(all_events[i][0], 3)
      all_events[i][1] = round(all_events[i][1], 3)

    all_events.sort(key=lambda x: x[0])

    see = all_events
    n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

    with open(n_label, 'w') as fp:
      fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

In [ ]:
def extract_labels_2(annotation_path):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]

  n_label = "/content/eval-files-2/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [ ]:
# shutil.rmtree('/content/eval-files-2/')
os.mkdir("/content/eval-files-2/")

In [ ]:
for audio in val_files:
  extract_labels_2(audio.replace(".wav", ".txt").replace("audio", "annotations"))

In [ ]:
import time

class MyCustomCallback_44(tf.keras.callbacks.Callback):
  def __init__(self):
    super(MyCustomCallback_44, self).__init__()
    self.best_f1 = 0.0
    self.best_error = np.inf


  def on_train_begin(self, logs=None):
    pass

  def on_train_end(self, logs=None):
    pass

  def on_epoch_end(self, epoch, logs=None):
    if True:
      start_time = time.time()

      # for ii, audio in enumerate(val_files):
      #   audio_file_path = audio
      #   see = mk_preds_YOHO_mel(self.model, ii)
      #   n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

      #   with open(n_label, 'w') as fp:
      #     fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

      mk_preds_YOHO_mel_2(self.model, mss_ins=mss_ins_2)

      stop_time = time.time()
      # print("Time for prediction: {}".format(stop_time - start_time))

      start_time = time.time()
      destination = "/content/eval-files-2/"
      test_set = glob.glob(destination + "*[0-9].txt")

      eval_path = "/content/"


      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt")
          }
          for tt in test_set
      ]

      data = []

      # Get used event labels
      all_data = dcase_util.containers.MetaDataContainer()
      for file_pair in file_list:
          reference_event_list = sed_eval.io.load_event_list(
              filename=file_pair['reference_file']
          )
          estimated_event_list = sed_eval.io.load_event_list(
              filename=file_pair['estimated_file']
          )

          data.append({'reference_event_list': reference_event_list,
                      'estimated_event_list': estimated_event_list})

          all_data += reference_event_list

      event_labels = all_data.unique_event_labels

      # Start evaluating

      # Create metrics classes, define parameters
      segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
          event_label_list=event_labels,
          time_resolution=1.0
      )

      event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
          event_label_list=event_labels,
          t_collar=1.0
      )

      # Go through files
      for file_pair in data:
          segment_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

          event_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

      # Get only certain metrics
      overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
      curr_f1 = overall_segment_based_metrics['f_measure']['f_measure']
      curr_error = overall_segment_based_metrics['error_rate']['error_rate']

      if curr_f1 > self.best_f1:
        self.best_f1 = curr_f1
        self.model.save_weights("/content/model-best-f1.h5")

      if curr_error < self.best_error:
        self.best_error = curr_error
        self.model.save_weights("/content/model-best-error.h5")

      print("F-measure: {:.3f} vs {:.3f}".format(curr_f1, self.best_f1))
      print("Error rate: {:.3f} vs {:.3f}".format(curr_error, self.best_error))

      stop_time = time.time()

      # print("Time to calculate metrics: {}".format(stop_time - start_time))

      # Or print all metrics as reports

In [ ]:

class MyCustomCallback_CRNN(tf.keras.callbacks.Callback):
  def __init__(self):
    super(MyCustomCallback_CRNN, self).__init__()
    self.best_f1 = 0.0
    self.best_error = np.inf


  def on_train_begin(self, logs=None):
    pass

  def on_train_end(self, logs=None):
    pass

  def on_epoch_end(self, epoch, logs=None):
    if epoch > 5:
      start_time = time.time()

      # for ii, audio in enumerate(val_files):
      #   audio_file_path = audio
      #   see = mk_preds_CRNN_mel(self.model, ii)
      #   n_label = n_label = "/content/eval-files-2/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.ann"

      #   with open(n_label, 'w') as fp:
      #     fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))


      mk_preds_CRNN_mel_2(self.model, mss_ins=mss_ins_2)
      stop_time = time.time()
      print("Time for prediction: {}".format(stop_time - start_time))

      start_time = time.time()


      destination = "/content/eval-files-2/"
      test_set = glob.glob(destination + "*[0-9].txt")

      eval_path = "/content/"


      file_list = [
          {
          'reference_file': tt,
          'estimated_file': tt.replace(".txt","-se-prediction.txt")
          }
          for tt in test_set
      ]

      data = []

      # Get used event labels
      all_data = dcase_util.containers.MetaDataContainer()
      for file_pair in file_list:
          reference_event_list = sed_eval.io.load_event_list(
              filename=file_pair['reference_file']
          )
          estimated_event_list = sed_eval.io.load_event_list(
              filename=file_pair['estimated_file']
          )

          data.append({'reference_event_list': reference_event_list,
                      'estimated_event_list': estimated_event_list})

          all_data += reference_event_list

      event_labels = all_data.unique_event_labels

      # Start evaluating

      # Create metrics classes, define parameters
      segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
          event_label_list=event_labels,
          time_resolution=1.0
      )

      event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
          event_label_list=event_labels,
          t_collar=1.0
      )

      # Go through files
      for file_pair in data:
          segment_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

          event_based_metrics.evaluate(
              reference_event_list=file_pair['reference_event_list'],
              estimated_event_list=file_pair['estimated_event_list']
          )

      # Get only certain metrics
      overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
      curr_f1 = overall_segment_based_metrics['f_measure']['f_measure']
      curr_error = overall_segment_based_metrics['error_rate']['error_rate']

      if curr_f1 > self.best_f1:
        self.best_f1 = curr_f1
        self.model.save_weights("/content/model-best-f1.h5")

      if curr_error < self.best_error:
        self.best_error = curr_error
        self.model.save_weights("/content/model-best-error.h5")

      print("F-measure: {:.3f} vs {:.3f}".format(curr_f1, self.best_f1))
      print("Error rate: {:.3f} vs {:.3f}".format(curr_error, self.best_error))

      # Or print all metrics as reports

      stop_time = time.time()
      print("Time to calculate metrics: {}".format(stop_time - start_time))

In [ ]:
overall_segment_based_metrics['error_rate']['error_rate']

0.6128038897893031

In [ ]:
LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    ([3, 3], 1,   64),
    ([3, 3], 2,  128),
    ([3, 3], 1,  128),
    ([3, 3], 2,  256),
    ([3, 3], 1,  256),
    ([3, 3], 2,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 1,  512),
    ([3, 3], 2, 1024),
    ([3, 3], 1, 1024),
    ([3, 3], 1, 512),
    ([3, 3], 1, 256),
    ([3, 3], 1, 128),
    # ([3, 3], 1, 128),
    # ([3, 3], 1, 128)
]

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
from keras.regularizers import l2

In [ ]:
"""
Manually define YamNet
"""

# params = yamnet_params.Params()
m_features = tf.keras.Input(shape=(1001, 40), name="mel_input")
X = m_features
X = tf.keras.layers.Reshape((1001, 40, 1))(X)
X = tf.keras.layers.Conv2D(filters = 32, kernel_size=[3, 3], strides=2, padding='same', use_bias=False,
                           activation=None, name = "layer1/conv",
                             kernel_regularizer=l2(1e-3), bias_regularizer=l2(1e-3))(X)
X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer1/bn")(X)
X = tf.keras.layers.ReLU(name="layer1/relu")(X)

# X = tf.keras.layers.SpatialDropout2D(0.5)(X)

for i in range(len(LAYER_DEFS)):
  X = tf.keras.layers.DepthwiseConv2D(kernel_size=LAYER_DEFS[i][0], strides = LAYER_DEFS[i][1], depth_multiplier=1, padding='same', use_bias=False,
                                      activation=None, name="layer"+ str(i + 2)+"/depthwise_conv")(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/depthwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/depthwise_conv/relu")(X)
  X = tf.keras.layers.Conv2D(filters = LAYER_DEFS[i][2], kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None,
                             name = "layer"+ str(i + 2)+"/pointwise_conv",
                             kernel_regularizer=l2(1e-2), bias_regularizer=l2(1e-2))(X)
  X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer"+ str(i + 2)+"/pointwise_conv/bn")(X)
  X = tf.keras.layers.ReLU(name="layer"+ str(i + 2)+"/pointwise_conv/relu")(X)

  X = tf.keras.layers.SpatialDropout2D(0.1)(X)

  # X = tf.keras.layers.Dropout(0.1)(X)


# X = tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides = 1, depth_multiplier=1, padding='same', use_bias=False, activation=None, name="layer2/depthwise_conv")(X)
# X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer2/depthwise_conv/bn")(X)
# X = tf.keras.layers.ReLU(name="layer2/depthwise_conv/relu")(X)
# X = tf.keras.layers.Conv2D(filters = 64, kernel_size=[1, 1], strides=1, padding='same', use_bias=False, activation=None, name = "layer2/pointwise_conv")(X)
# X = tf.keras.layers.BatchNormalization(center=True, scale=False, epsilon=1e-4, name = "layer2/pointwise_conv/bn")(X)
# X = tf.keras.layers.ReLU(name="layer2/pointwise_conv/relu")(X)

_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)
# X = tf.keras.layers.Dropout(0.4)(X)

# X = _separable_conv("extra", [3, 3], 2, 1024, params)(predictions)
# X = tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-2))(X)
# X = layers.Dense(2, name="speech_and_music", activation='sigmoid')(embeddings)
# X = tf.keras.layers.Flatten()(X)
pred = tf.keras.layers.Conv1D(30,kernel_size=1, activation="sigmoid")(X)
# pred = tf.keras.layers.Dense(6, activation="sigmoid")(X)
model = tf.keras.Model(
      name='yamnet_frames', inputs=m_features,
      outputs=[pred])

In [ ]:
# models = []
# for i in range(4):
#   models.append(model)

In [ ]:
model.summary()

Model: "yamnet_frames"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel_input (InputLayer)       [(None, 1001, 40)]        0         
_________________________________________________________________
reshape_12 (Reshape)         (None, 1001, 40, 1)       0         
_________________________________________________________________
layer1/conv (Conv2D)         (None, 501, 20, 32)       288       
_________________________________________________________________
layer1/bn (BatchNormalizatio (None, 501, 20, 32)       96        
_________________________________________________________________
layer1/relu (ReLU)           (None, 501, 20, 32)       0         
_________________________________________________________________
layer2/depthwise_conv (Depth (None, 501, 20, 32)       288       
_________________________________________________________________
layer2/depthwise_conv/bn (Ba (None, 501, 20, 32)     

In [ ]:
model.load_weights("/content/drive/MyDrive/Urban-SED/Models/model-best-error-YamNet.h5")

# Define the CRNN network

In [ ]:
"""
The CRNN model used for ICASSP
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

mel_input = keras.Input(shape=(257, 40), name="mel_input")
X = mel_input

X = tf.keras.layers.Reshape((257, 40, 1))(X)
# print(X.shape)

X = tf.keras.layers.Conv2D(filters=16, kernel_size=7, strides=1, padding='same')(X)
X = layers.BatchNormalization(momentum=0.0)(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = tf.keras.layers.Dropout(rate = 0.2)(X)

X = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=1, padding='same')(X)
X = layers.BatchNormalization(momentum=0.0)(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = layers.Dropout(rate = 0.2)(X)

print(X.shape)
_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)

X = layers.Bidirectional(layers.GRU(80, return_sequences = True))(X)
X = layers.BatchNormalization(momentum=0.0)(X)

X = layers.Bidirectional(layers.GRU(80, return_sequences = True))(X)
X = layers.BatchNormalization(momentum=0.0)(X)

pred = layers.TimeDistributed(layers.Dense(6, activation='sigmoid'))(X)

model = keras.Model(inputs = [mel_input], outputs = [pred])

keras.utils.plot_model(model, "CRNN.png", show_shapes=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=[keras.losses.BinaryCrossentropy()], metrics=['binary_accuracy'] #, 'categorical_accuracy', tf.keras.metrics.Precision(class_id=0), tf.keras.metrics.Precision(class_id=1), tf.keras.metrics.Recall(class_id=0), tf.keras.metrics.Recall(class_id=1)]
)

model.summary()

(None, 257, 10, 64)
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel_input (InputLayer)       [(None, 257, 40)]         0         
_________________________________________________________________
reshape_8 (Reshape)          (None, 257, 40, 1)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 257, 40, 16)       800       
_________________________________________________________________
batch_normalization_12 (Batc (None, 257, 40, 16)       64        
_________________________________________________________________
activation_12 (Activation)   (None, 257, 40, 16)       0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 257, 20, 16)       0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 257

In [ ]:
"""
The CRNN model used by Adavanne et al. 2017 for DCASE Task 3
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

mel_input = keras.Input(shape=(257, 40), name="mel_input")
X = mel_input

X = tf.keras.layers.Reshape((257, 40, 1))(X)
# print(X.shape)

X = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 5))(X)
X = tf.keras.layers.Dropout(rate = 0.05)(X)

X = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = layers.Dropout(rate = 0.05)(X)

X = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = layers.Dropout(rate = 0.05)(X)

print(X.shape)
_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)

X = layers.Bidirectional(layers.GRU(32, return_sequences = True))(X)
# X = layers.BatchNormalization()(X)
X = layers.Dropout(rate = 0.05)(X)

X = layers.Bidirectional(layers.GRU(32, return_sequences = True))(X)
# X = layers.BatchNormalization()(X)
X = layers.Dropout(rate = 0.05)(X)

X = layers.TimeDistributed(layers.Dense(16))(X)

pred = layers.TimeDistributed(layers.Dense(6, activation='sigmoid'))(X)

model = keras.Model(inputs = [mel_input], outputs = [pred])

keras.utils.plot_model(model, "CRNN.png", show_shapes=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=[keras.losses.BinaryCrossentropy()], metrics=['binary_accuracy'] #, 'categorical_accuracy', tf.keras.metrics.Precision(class_id=0), tf.keras.metrics.Precision(class_id=1), tf.keras.metrics.Recall(class_id=0), tf.keras.metrics.Recall(class_id=1)]
)

model.summary()

(None, 257, 2, 128)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel_input (InputLayer)       [(None, 257, 40)]         0         
_________________________________________________________________
reshape (Reshape)            (None, 257, 40, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 257, 40, 128)      1280      
_________________________________________________________________
batch_normalization (BatchNo (None, 257, 40, 128)      512       
_________________________________________________________________
activation (Activation)      (None, 257, 40, 128)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 257, 8, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 257, 

In [ ]:
"""
The CRNN model suggested by Justin Salamon
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

mel_input = keras.Input(shape=(1001, 40), name="mel_input")
X = mel_input

X = tf.keras.layers.Reshape((1001, 40, 1))(X)
# print(X.shape)

X = tf.keras.layers.Conv2D(filters=96, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = tf.keras.layers.Dropout(rate = 0.25)(X)

X = tf.keras.layers.Conv2D(filters=96, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = layers.Dropout(rate = 0.25)(X)

X = tf.keras.layers.Conv2D(filters=96, kernel_size=3, strides=1, padding='same')(X)
X = layers.BatchNormalization()(X)
X = tf.keras.layers.Activation('relu')(X)
X = tf.keras.layers.MaxPool2D(pool_size=(1, 2))(X)
X = layers.Dropout(rate = 0.25)(X)

print(X.shape)
_, _, sx, sy = X.shape
X = tf.keras.layers.Reshape((-1, int(sx * sy)))(X)

X = layers.Bidirectional(layers.GRU(96, return_sequences = True))(X)
# X = layers.BatchNormalization()(X)
# X = layers.Dropout(rate = 0.05)(X)

X = layers.Bidirectional(layers.GRU(96, return_sequences = True))(X)
# X = layers.BatchNormalization()(X)
# X = layers.Dropout(rate = 0.05)(X)

X = layers.Bidirectional(layers.GRU(96, return_sequences = True))(X)
# X = layers.BatchNormalization()(X)
# X = layers.Dropout(rate = 0.05)(X)

# X = layers.TimeDistributed(layers.Dense(16))(X)

pred = layers.TimeDistributed(layers.Dense(10, activation='sigmoid'))(X)

model = keras.Model(inputs = [mel_input], outputs = [pred])

keras.utils.plot_model(model, "CRNN.png", show_shapes=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=[keras.losses.BinaryCrossentropy()], metrics=['binary_accuracy'] #, 'categorical_accuracy', tf.keras.metrics.Precision(class_id=0), tf.keras.metrics.Precision(class_id=1), tf.keras.metrics.Recall(class_id=0), tf.keras.metrics.Recall(class_id=1)]
)

model.summary()

(None, 1001, 5, 96)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel_input (InputLayer)       [(None, 1001, 40)]        0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 1001, 40, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1001, 40, 96)      960       
_________________________________________________________________
batch_normalization_3 (Batch (None, 1001, 40, 96)      384       
_________________________________________________________________
activation_3 (Activation)    (None, 1001, 40, 96)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1001, 20, 96)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100

In [ ]:
# models = []
# for i in range(4):
#   models.append(model)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [ ]:
m_auc = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve="ROC",
    summation_method="interpolation",
    multi_label=True,
    num_labels=162,
    # label_weights=[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    from_logits=False,
)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=250,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
model.compile(optimizer=optimizer, loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=my_loss_fn, metrics=[binary_acc])

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_44()], verbose=2)

Epoch 1/1000
187/187 - 86s - loss: 81.6531 - binary_acc: 0.9156 - val_loss: 103.4722 - val_binary_acc: 0.9186
Epoch 2/1000
187/187 - 27s - loss: 58.9936 - binary_acc: 0.9220 - val_loss: 78.3432 - val_binary_acc: 0.9215
Epoch 3/1000
187/187 - 27s - loss: 46.0800 - binary_acc: 0.9244 - val_loss: 53.9265 - val_binary_acc: 0.9215
F-measure: nan vs 0.000
Error rate: 1.000 vs 1.000
Epoch 4/1000
187/187 - 27s - loss: 38.6019 - binary_acc: 0.9267 - val_loss: 40.2818 - val_binary_acc: 0.9214
F-measure: 0.002 vs 0.002
Error rate: 0.999 vs 0.999
Epoch 5/1000
187/187 - 27s - loss: 33.7961 - binary_acc: 0.9290 - val_loss: 36.6477 - val_binary_acc: 0.9101
F-measure: 0.264 vs 0.264
Error rate: 0.880 vs 0.880
Epoch 6/1000
187/187 - 27s - loss: 30.4342 - binary_acc: 0.9315 - val_loss: 37.6812 - val_binary_acc: 0.8933
F-measure: 0.325 vs 0.325
Error rate: 0.894 vs 0.880
Epoch 7/1000
187/187 - 27s - loss: 28.1055 - binary_acc: 0.9328 - val_loss: 31.7085 - val_binary_acc: 0.9131
F-measure: 0.379 vs 0.379


KeyboardInterrupt: ignored

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_CRNN()], verbose=2)

Epoch 1/1000
187/187 - 97s - loss: 0.2125 - binary_accuracy: 0.9271 - val_loss: 0.2727 - val_binary_accuracy: 0.9141
Epoch 2/1000
187/187 - 69s - loss: 0.1814 - binary_accuracy: 0.9330 - val_loss: 0.2724 - val_binary_accuracy: 0.8935
Epoch 3/1000
187/187 - 69s - loss: 0.1669 - binary_accuracy: 0.9364 - val_loss: 0.1783 - val_binary_accuracy: 0.9325
Epoch 4/1000
187/187 - 69s - loss: 0.1576 - binary_accuracy: 0.9392 - val_loss: 0.2767 - val_binary_accuracy: 0.8775
Epoch 5/1000
187/187 - 69s - loss: 0.1496 - binary_accuracy: 0.9420 - val_loss: 0.1762 - val_binary_accuracy: 0.9323
Epoch 6/1000
187/187 - 69s - loss: 0.1437 - binary_accuracy: 0.9440 - val_loss: 0.2170 - val_binary_accuracy: 0.9112
Epoch 7/1000
187/187 - 69s - loss: 0.1376 - binary_accuracy: 0.9462 - val_loss: 0.2079 - val_binary_accuracy: 0.9139
Time for prediction: 102.12985944747925
F-measure: 0.408 vs 0.408
Error rate: 0.878 vs 0.878
Time to calculate metrics: 10.60545802116394
Epoch 8/1000
187/187 - 69s - loss: 0.1356 -

KeyboardInterrupt: ignored

In [ ]:
!cp "/content/model-best-f1.h5" "/content/drive/MyDrive/Urban-SED/Models/CRNN/"

In [ ]:
model.load_weights("/content/model-best-error.h5")

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_CRNN()], verbose=2, initial_epoch=9)

Epoch 10/1000
187/187 - 69s - loss: 0.1256 - binary_accuracy: 0.9507 - val_loss: 0.1768 - val_binary_accuracy: 0.9313
Time for prediction: 102.58195924758911
F-measure: nan vs 0.000
Error rate: 0.000 vs 0.000
Time to calculate metrics: 0.06642889976501465
Epoch 11/1000
187/187 - 69s - loss: 0.1206 - binary_accuracy: 0.9528 - val_loss: 0.1759 - val_binary_accuracy: 0.9327
Time for prediction: 101.06238079071045
F-measure: nan vs 0.000
Error rate: 0.000 vs 0.000
Time to calculate metrics: 0.00789189338684082
Epoch 12/1000
187/187 - 69s - loss: 0.1177 - binary_accuracy: 0.9540 - val_loss: 0.1835 - val_binary_accuracy: 0.9325
Time for prediction: 98.81635332107544
F-measure: nan vs 0.000
Error rate: 0.000 vs 0.000
Time to calculate metrics: 0.009307622909545898
Epoch 13/1000
187/187 - 69s - loss: 0.1120 - binary_accuracy: 0.9562 - val_loss: 0.1900 - val_binary_accuracy: 0.9269
Time for prediction: 97.24101066589355
F-measure: nan vs 0.000
Error rate: 0.000 vs 0.000
Time to calculate metric

KeyboardInterrupt: ignored

In [ ]:
shutil.copy2("/content/model-best-error-SAYamNet.h5", "/content/drive/MyDrive/Urban-SED/Models/")

'/content/drive/MyDrive/Urban-SED/Models/model-best-error-SAYamNet.h5'

In [ ]:
mss_ins.shape

(2000,)

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_CRNN()], verbose=1)

Epoch 1/1000
67/67 [==============================] - 12s 93ms/step - loss: 0.4096 - binary_accuracy: 0.8321 - val_loss: 0.4934 - val_binary_accuracy: 0.7679
Epoch 2/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.3413 - binary_accuracy: 0.8655 - val_loss: 0.5049 - val_binary_accuracy: 0.7660
Epoch 3/1000
67/67 [==============================] - 5s 77ms/step - loss: 0.3268 - binary_accuracy: 0.8701 - val_loss: 0.4862 - val_binary_accuracy: 0.7943
F-measure: 0.261 vs 0.261
Error rate: 0.848 vs 0.848
Epoch 4/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.3123 - binary_accuracy: 0.8739 - val_loss: 0.5001 - val_binary_accuracy: 0.7942
F-measure: 0.279 vs 0.279
Error rate: 0.842 vs 0.842
Epoch 5/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.3067 - binary_accuracy: 0.8757 - val_loss: 0.5263 - val_binary_accuracy: 0.7952
F-measure: 0.326 vs 0.326
Error rate: 0.797 vs 0.797
Epoch 6/1000
67/67 [==============================] - 5

KeyboardInterrupt: ignored

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_CRNN()], verbose=1)

Epoch 1/1000
67/67 [==============================] - 15s 111ms/step - loss: 0.4103 - binary_accuracy: 0.8269 - val_loss: 0.3966 - val_binary_accuracy: 0.8080
Epoch 2/1000
67/67 [==============================] - 6s 88ms/step - loss: 0.3735 - binary_accuracy: 0.8416 - val_loss: 0.3802 - val_binary_accuracy: 0.8371
Epoch 3/1000
67/67 [==============================] - 6s 88ms/step - loss: 0.3514 - binary_accuracy: 0.8497 - val_loss: 0.3997 - val_binary_accuracy: 0.8328
F-measure: 0.373 vs 0.373
Error rate: 1.509 vs 1.509
Epoch 4/1000
67/67 [==============================] - 6s 88ms/step - loss: 0.3371 - binary_accuracy: 0.8584 - val_loss: 0.3762 - val_binary_accuracy: 0.8444
F-measure: 0.438 vs 0.438
Error rate: 0.908 vs 0.908
Epoch 5/1000
67/67 [==============================] - 6s 87ms/step - loss: 0.3277 - binary_accuracy: 0.8612 - val_loss: 0.3539 - val_binary_accuracy: 0.8681
F-measure: 0.435 vs 0.438
Error rate: 1.016 vs 0.908
Epoch 6/1000
67/67 [==============================] - 

KeyboardInterrupt: ignored

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=1000, callbacks=[MyCustomCallback_CRNN()], verbose=1)

Epoch 1/1000
67/67 [==============================] - 15s 101ms/step - loss: 0.5701 - binary_accuracy: 0.7352 - val_loss: 0.5373 - val_binary_accuracy: 0.7979
Epoch 2/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.3754 - binary_accuracy: 0.8735 - val_loss: 0.3424 - val_binary_accuracy: 0.8758
Epoch 3/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.2840 - binary_accuracy: 0.8931 - val_loss: 0.3601 - val_binary_accuracy: 0.8737
F-measure: 0.468 vs 0.468
Error rate: 0.923 vs 0.923
Epoch 4/1000
67/67 [==============================] - 5s 77ms/step - loss: 0.2472 - binary_accuracy: 0.9023 - val_loss: 0.3792 - val_binary_accuracy: 0.8565
F-measure: 0.452 vs 0.468
Error rate: 1.141 vs 0.923
Epoch 5/1000
67/67 [==============================] - 5s 78ms/step - loss: 0.2272 - binary_accuracy: 0.9081 - val_loss: 0.3327 - val_binary_accuracy: 0.8788
F-measure: 0.494 vs 0.494
Error rate: 0.880 vs 0.880
Epoch 6/1000
67/67 [==============================] - 

KeyboardInterrupt: ignored

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=300, callbacks=[callback])

Epoch 1/300
67/67 [==============================] - 7s 56ms/step - loss: 9.5532 - binary_acc: 0.8514 - val_loss: 15.7825 - val_binary_acc: 0.7547
Epoch 2/300
67/67 [==============================] - 3s 44ms/step - loss: 7.8473 - binary_acc: 0.8609 - val_loss: 16.3404 - val_binary_acc: 0.7368
Epoch 3/300
67/67 [==============================] - 3s 45ms/step - loss: 8.0458 - binary_acc: 0.8593 - val_loss: 21.4179 - val_binary_acc: 0.5977
Epoch 4/300
67/67 [==============================] - 3s 44ms/step - loss: 7.4105 - binary_acc: 0.8640 - val_loss: 25.0913 - val_binary_acc: 0.5850
Epoch 5/300
67/67 [==============================] - 3s 44ms/step - loss: 8.2033 - binary_acc: 0.8618 - val_loss: 13.0090 - val_binary_acc: 0.7592
Epoch 6/300
67/67 [==============================] - 3s 44ms/step - loss: 7.8684 - binary_acc: 0.8648 - val_loss: 9.3664 - val_binary_acc: 0.8557
Epoch 7/300
67/67 [==============================] - 3s 44ms/step - loss: 7.3047 - binary_acc: 0.8705 - val_loss: 7.193

KeyboardInterrupt: ignored

In [ ]:
model.save_weights("/content/drive/MyDrive/TUT-sound-events-2017/Models/YamNet-1.h5")

In [ ]:
model.evaluate(validation_generator)

20/20 [==============================] - 1s 32ms/step - loss: 8.0318 - binary_acc: 0.8648


[8.031794548034668, 0.8648437261581421]

In [ ]:
model.fit(training_generator, validation_data=validation_generator, epochs=200, callbacks=[callback])

Epoch 1/200
16/16 [==============================] - 11s 324ms/step - loss: 0.6686 - binary_accuracy: 0.7024 - val_loss: 0.4066 - val_binary_accuracy: 0.8119
Epoch 2/200
16/16 [==============================] - 4s 262ms/step - loss: 0.3484 - binary_accuracy: 0.8521 - val_loss: 0.3416 - val_binary_accuracy: 0.8756
Epoch 3/200
16/16 [==============================] - 4s 262ms/step - loss: 0.3339 - binary_accuracy: 0.8576 - val_loss: 0.3575 - val_binary_accuracy: 0.8756
Epoch 4/200
16/16 [==============================] - 4s 262ms/step - loss: 0.3115 - binary_accuracy: 0.8666 - val_loss: 0.3071 - val_binary_accuracy: 0.8897
Epoch 5/200
16/16 [==============================] - 4s 263ms/step - loss: 0.3057 - binary_accuracy: 0.8711 - val_loss: 0.3479 - val_binary_accuracy: 0.8772
Epoch 6/200
16/16 [==============================] - 4s 263ms/step - loss: 0.2982 - binary_accuracy: 0.8739 - val_loss: 0.3577 - val_binary_accuracy: 0.8820
Epoch 7/200
16/16 [==============================] - 4s 2

In [ ]:
model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), epochs=100, callbacks=[callback])

Epoch 1/100
17/17 [==============================] - 28s 352ms/step - loss: 0.7176 - binary_accuracy: 0.6044 - val_loss: 0.6490 - val_binary_accuracy: 0.6814
Epoch 2/100
17/17 [==============================] - 5s 268ms/step - loss: 0.5888 - binary_accuracy: 0.7342 - val_loss: 0.5936 - val_binary_accuracy: 0.7427
Epoch 3/100
17/17 [==============================] - 5s 271ms/step - loss: 0.5385 - binary_accuracy: 0.7947 - val_loss: 0.4798 - val_binary_accuracy: 0.8292
Epoch 4/100
17/17 [==============================] - 5s 273ms/step - loss: 0.4985 - binary_accuracy: 0.8176 - val_loss: 0.6254 - val_binary_accuracy: 0.7025
Epoch 5/100
17/17 [==============================] - 5s 275ms/step - loss: 0.4419 - binary_accuracy: 0.8572 - val_loss: 0.5325 - val_binary_accuracy: 0.8091
Epoch 6/100
17/17 [==============================] - 5s 272ms/step - loss: 0.3970 - binary_accuracy: 0.8746 - val_loss: 0.3979 - val_binary_accuracy: 0.8507
Epoch 7/100
17/17 [==============================] - 5s 2

In [ ]:
model.compile(metrics=tf.keras.metrics.Recall(class_id=0))

In [ ]:
model.evaluate(x=X_val, y=Y_val)

6/6 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - recall: 0.0000e+00


[0.0, 0.0]

In [ ]:
mel_files = glob.glob('/content/YamNet/**/*.*', recursive=True)
print(mel_files)
zip_file_name = "/content/drive/MyDrive/TUT-sound-events-2017/Models/29-6-21/YamNet.zip"

with ZipFile(zip_file_name, 'w') as my_zip:
  for f in mel_files:
    my_zip.write(f)


['/content/YamNet/seg eval YamNet-fold4.pickle', '/content/YamNet/model-best-f1.h5', '/content/YamNet/seg eval YamNet-fold1.pickle', '/content/YamNet/seg eval YamNet-fold3.pickle', '/content/YamNet/seg eval YamNet-fold4.txt', '/content/YamNet/model-best-error-YamNet-fold2.h5', '/content/YamNet/event eval YamNet-fold4.pickle', '/content/YamNet/event eval YamNet-fold3.pickle', '/content/YamNet/seg eval YamNet-fold2.txt', '/content/YamNet/event eval YamNet-fold4.txt', '/content/YamNet/event eval YamNet-fold1.pickle', '/content/YamNet/event eval YamNet-fold1.txt', '/content/YamNet/event eval YamNet-fold3.txt', '/content/YamNet/seg eval YamNet-fold2.pickle', '/content/YamNet/model-best-error-YamNet-fold1.h5', '/content/YamNet/model-best-error-YamNet-fold4.h5', '/content/YamNet/seg eval YamNet-fold3.txt', '/content/YamNet/seg eval YamNet-fold1.txt', '/content/YamNet/event eval YamNet-fold2.txt', '/content/YamNet/model-best-error-YamNet-fold3.h5', '/content/YamNet/event eval YamNet-fold2.pick

# Testing

In [ ]:
class C1F1(tf.keras.metrics.Metric):

  def __init__(self, name='c1_f1', **kwargs):
    super(C1F1, self).__init__(name=name, **kwargs)
    self.tp = self.add_weight(name='true_positive', initializer='zeros')
    self.fp = self.add_weight(name='false_positive', initializer='zeros')
    self.tn = self.add_weight(name='true_negative', initializer='zeros')
    self.fn = self.add_weight(name='false_negative', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):

    threshold = tf.constant([0.5])

    binary_true = y_true[:, :, 0]
    binary_pred = y_pred[:, :, 0]

    binary_true = tf.greater_equal(binary_true, threshold)
    binary_pred = tf.greater_equal(binary_pred, threshold)

    tp = tf.cast(tf.logical_and(tf.equal(binary_true, True), tf.equal(binary_pred, True)), dtype = np.float32)
    fp = tf.cast(tf.logical_and(tf.equal(binary_true, False), tf.equal(binary_pred, True)), dtype = np.float32)
    tn = tf.cast(tf.logical_and(tf.equal(binary_true, False), tf.equal(binary_pred, False)), dtype = np.float32)
    fn = tf.cast(tf.logical_and(tf.equal(binary_true, True), tf.equal(binary_pred, False)), dtype = np.float32)

    self.tp.assign_add(tf.reduce_sum(tp, axis = None))
    self.fp.assign_add(tf.reduce_sum(fp, axis = None))
    self.tn.assign_add(tf.reduce_sum(tn, axis = None))
    self.fn.assign_add(tf.reduce_sum(fn, axis = None))

  def result(self):
    binary_f1 = self.tp / (self.tp +  0.5 * (self.fp + self.fn))
    return binary_f1

  def reset_states(self):
    self.tp.assign(0)
    self.fp.assign(0)
    self.tn.assign(0)
    self.fn.assign(0)

class C2F1(tf.keras.metrics.Metric):

  def __init__(self, name='c2_f1', **kwargs):
    super(C2F1, self).__init__(name=name, **kwargs)
    self.tp = self.add_weight(name='true_positive', initializer='zeros')
    self.fp = self.add_weight(name='false_positive', initializer='zeros')
    self.tn = self.add_weight(name='true_negative', initializer='zeros')
    self.fn = self.add_weight(name='false_negative', initializer='zeros')

  def update_state(self, y_true, y_pred, sample_weight=None):

    threshold = tf.constant([0.5])

    binary_true = y_true[:, :, 0]
    binary_pred = y_pred[:, :, 0]

    binary_true = tf.greater_equal(binary_true, threshold)
    binary_pred = tf.greater_equal(binary_pred, threshold)

    tp = tf.cast(tf.logical_and(tf.equal(binary_true, True), tf.equal(binary_pred, True)), dtype = np.float32)
    fp = tf.cast(tf.logical_and(tf.equal(binary_true, False), tf.equal(binary_pred, True)), dtype = np.float32)
    tn = tf.cast(tf.logical_and(tf.equal(binary_true, False), tf.equal(binary_pred, False)), dtype = np.float32)
    fn = tf.cast(tf.logical_and(tf.equal(binary_true, True), tf.equal(binary_pred, False)), dtype = np.float32)

    self.tp.assign_add(tf.reduce_sum(tp, axis = None))
    self.fp.assign_add(tf.reduce_sum(fp, axis = None))
    self.tn.assign_add(tf.reduce_sum(tn, axis = None))
    self.fn.assign_add(tf.reduce_sum(fn, axis = None))

  def result(self):
    binary_f1 = self.tp / (self.tp +  0.5 * (self.fp + self.fn))
    return binary_f1

  def reset_states(self):
    self.tp.assign(0)
    self.fp.assign(0)
    self.tn.assign(0)
    self.fn.assign(0)

In [ ]:
def smoothe_events(events):

  ann = events

  curr_ann = ann

  class_set = set([c[2] for c in curr_ann])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in curr_ann:
    class_wise_events[c[2]].append(c)

  max_event_silence = 1.0
  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events

In [ ]:
rev_class_dict = ['brakes squeaking',
              'car',
              'children',
              'large vehicle',
              'people speaking',
              'people walking']

In [ ]:
def mk_preds_YOHO(model, audio_path, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((len(a), 9, 18))
  mss_in = np.zeros((len(a), 257, 40))

  for i in range(len(a)):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)):
      for jjj in range(0, 6):
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges[i][0]
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
def frames_to_time(f, sr = 44100.0, hop_size = 441):
  return f * hop_size / sr

def preds_to_se(p, win_start, audio_clip_length = 2.56):
  start_dicts = [-100, -100, -100, -100, -100, -100]
  stop_dicts = [-100, -100, -100, -100, -100, -100]


  start_speech = -100
  start_music = -100
  stop_speech = -100
  stop_music = -100

  audio_events = []

  n_frames = p.shape[0]

  for j in range(p.shape[1]):
    if p[0, j] >= 0.5:
      start_dicts[j] = 0

  for j in range(p.shape[1]):
    for i in range(n_frames - 1):
      if p[i, j] < 0.5 and p[i+1, j] >= 0.5:
        start_dicts[j] = i+1

      elif p[i, j] >= 0.5 and p[i + 1, j] < 0.5:
        stop_dicts[j] = i
        start_time = frames_to_time(start_dicts[j])
        stop_time = frames_to_time(stop_dicts[j])

        audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
        start_dicts[j] = -100
        stop_dicts[j] = -100

    if start_dicts[j] != -100:
      start_time = frames_to_time(start_dicts[j])
      stop_time = audio_clip_length
      audio_events.append([start_time+win_start, stop_time+win_start, rev_class_dict[j]])
      start_dicts[j] = -100
      stop_dicts[j] = -100

  audio_events.sort(key = lambda x: x[0])
  return audio_events



In [ ]:
def mk_preds_CRNN(model, audio_path, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((len(a), 9, 18))
  mss_in = np.zeros((len(a), 257, 40))

  for i in range(len(a)):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div

    events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
# in_signal, in_sr = sf.read("")

# Resample the audio file.
win_length = 2.56
no_of_div = 9

audio_clip_length_samples = in_signal.shape[0]
print('audio_clip_length_samples is {}'.format(audio_clip_length_samples))

a, win_ranges = construct_examples("/content/development/TUT-sound-events-2017-development/audio-mono/street/a001.wav",hop_len=1.96)

preds = np.zeros((len(a), 9, 18))
mss_in = np.zeros((len(a), 257, 40))

for i in range(len(a)):
  M = get_log_melspectrogram(a[i])
  mss_in[i, :, :] = M.T

preds = model.predict(mss_in)
events = []

for i in range(len(preds)):
  p = preds[i, :, :]
  events_curr = []
  win_width = win_length / no_of_div
  for j in range(len(p)):
    for jjj in range(0, 6):
      if p[j][jjj*3] >= 0.5:
        start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges[i][0]
        end = p[j][jjj*3+2] * win_width + start
        events_curr.append([start, end, rev_class_dict[jjj]])

  events += events_curr


class_set = set([c[2] for c in events])
class_wise_events = {}

for c in list(class_set):
  class_wise_events[c] = []


for c in events:
  class_wise_events[c[2]].append(c)

max_event_silence = 1.0
all_events = []

for k in list(class_wise_events.keys()):
  curr_events = class_wise_events[k]
  count = 0

  while count < len(curr_events) - 1:
    if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
      curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
      del curr_events[count + 1]
    else:
      count += 1

  all_events += curr_events

for i in range(len(all_events)):
  all_events[i][0] = round(all_events[i][0], 3)
  all_events[i][1] = round(all_events[i][1], 3)

all_events.sort(key=lambda x: x[0])


NameError: ignored

In [ ]:
all_events

[[3.931, 16.254, 'car'],
 [22.13, 22.69, 'large vehicle'],
 [24.09, 28.02, 'large vehicle'],
 [27.456, 29.993, 'people walking'],
 [29.403, 33.057, 'large vehicle'],
 [31.374, 31.642, 'people walking'],
 [35.568, 36.976, 'large vehicle'],
 [41.161, 42.288, 'large vehicle'],
 [45.095, 45.924, 'car'],
 [47.042, 51.804, 'large vehicle'],
 [52.921, 53.201, 'large vehicle'],
 [54.881, 55.161, 'large vehicle'],
 [58.802, 68.052, 'large vehicle'],
 [70.575, 71.406, 'car'],
 [72.536, 72.808, 'car'],
 [75.055, 75.324, 'car'],
 [121.538, 129.929, 'car'],
 [131.335, 131.889, 'car'],
 [137.215, 138.042, 'car'],
 [139.172, 151.209, 'car'],
 [160.737, 165.485, 'car'],
 [166.612, 167.441, 'car'],
 [168.575, 173.321, 'car'],
 [174.456, 175.01, 'car']]

In [ ]:
events

[[3.931, 4.209, 'car'],
 [4.216, 4.763, 'car'],
 [4.490297043455971, 4.763090469969643, 'car'],
 [4.774, 5.044, 'car'],
 [5.058, 5.325, 'car'],
 [5.888, 6.168, 'car'],
 [6.172, 6.726, 'car'],
 [6.449815937280655, 6.726452103323407, 'car'],
 [6.734, 7.009, 'car'],
 [7.018, 7.287, 'car'],
 [7.302, 7.569, 'car'],
 [7.587, 7.854, 'car'],
 [7.871, 8.41, 'car'],
 [7.847866964870029, 8.128099797036914, 'car'],
 [8.13259925895267, 8.41000141620636, 'car'],
 [8.41, 8.684, 'car'],
 [8.694, 8.965, 'car'],
 [8.978, 9.241, 'car'],
 [9.263, 9.525, 'car'],
 [9.814, 10.089, 'car'],
 [10.098, 10.644, 'car'],
 [10.371614213784536, 10.644089172416264, 'car'],
 [10.654, 10.923, 'car'],
 [11.775, 12.048, 'car'],
 [12.059, 12.332, 'car'],
 [12.333, 12.608, 'car'],
 [12.615, 12.886, 'car'],
 [12.899, 13.159, 'car'],
 [13.183, 13.445, 'car'],
 [13.73, 14.009, 'car'],
 [14.014, 14.291, 'car'],
 [14.292, 14.565, 'car'],
 [14.574, 14.838, 'car'],
 [14.858, 15.11, 'car'],
 [15.696, 15.969, 'car'],
 [15.982, 16.25

In [ ]:
a[0].shape[0]/44100

2.56

In [ ]:
np.array(a).shape

(94, 112896)

In [ ]:
win_ranges

[(0.0, 2.56),
 (1.96, 4.52),
 (3.92, 6.48),
 (5.88, 8.44),
 (7.84, 10.4),
 (9.8, 12.36),
 (11.76, 14.32),
 (13.72, 16.28),
 (15.68, 18.24),
 (17.64, 20.2),
 (19.6, 22.16),
 (21.56, 24.12),
 (23.52, 26.08),
 (25.48, 28.04),
 (27.44, 30.0),
 (29.4, 31.96),
 (31.36, 33.92),
 (33.32, 35.88),
 (35.28, 37.84),
 (37.24, 39.8),
 (39.2, 41.76),
 (41.16, 43.72),
 (43.12, 45.68),
 (45.08, 47.64),
 (47.04, 49.6),
 (49.0, 51.56),
 (50.96, 53.52),
 (52.92, 55.48),
 (54.88, 57.44),
 (56.84, 59.4),
 (58.8, 61.36),
 (60.76, 63.32),
 (62.72, 65.28),
 (64.68, 67.24),
 (66.64, 69.2),
 (68.6, 71.16),
 (70.56, 73.12),
 (72.52, 75.08),
 (74.48, 77.04),
 (76.44, 79.0),
 (78.4, 80.96),
 (80.36, 82.92),
 (82.32, 84.88),
 (84.28, 86.84),
 (86.24, 88.8),
 (88.2, 90.76),
 (90.16, 92.72),
 (92.12, 94.68),
 (94.08, 96.64),
 (96.04, 98.6),
 (98.0, 100.56),
 (99.96, 102.52),
 (101.92, 104.48),
 (103.88, 106.44),
 (105.84, 108.4),
 (107.8, 110.36),
 (109.76, 112.32),
 (111.72, 114.28),
 (113.68, 116.24),
 (115.64, 118.

In [ ]:
"""
Make predictions for full audio --- vectorised implementation.
"""

def mk_preds_vector(audio_path, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, sampling_rate = 44100):
  in_signal, in_sr = sf.read(audio_path)

  # Resample the audio file.


  audio_clip_length_samples = in_signal.shape[0]
  print('audio_clip_length_samples is {}'.format(audio_clip_length_samples))

  hop_size_samples = int(hop_size * sampling_rate)
  # hop_size_samples = 220 * 602 - 1

  win_length_samples = int(win_length * sampling_rate)
  # win_length_samples = 220 * 802 - 1

  n_preds = int(math.ceil((audio_clip_length_samples - win_length_samples) / hop_size_samples)) + 1

  # n_preds = int()

  #print('n_preds is {}'.format(n_preds))

  in_signal_pad = np.zeros(((n_preds - 1) * hop_size_samples) + win_length_samples)
  # in_signal_pad = np.zeros((n_preds * hop_size_samples + 200 * 220))

  #print('in_signal_pad.shape is {}'.format(in_signal_pad.shape))

  in_signal_pad[0:audio_clip_length_samples] = in_signal

  preds = np.zeros((n_preds, 9, 18))
  mss_in = np.zeros((n_preds, 257, 40))
  events = []

  for i in range(n_preds):
    seg = in_signal_pad[i * hop_size_samples:(i * hop_size_samples) + win_length_samples]
    #print('seg.shape is {}'.format(seg.shape))

    mss = get_log_melspectrogram(seg)
    M = mss.T
    mss_in[i, :, :] = M

  preds = model.predict(mss_in)
  # preds[:, 0] = (p[:, 0] >= 0.5).astype(np.float)
  # preds[:, 2] = (p[:, 2] >= 0.5).astype(np.float)

  events = []

  for j in range(n_preds):
    p = preds[j, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for i in range(len(p)):
      for jjj in range(0, 6):
        if p[i][jjj*3] >= 0.5:
          start = win_width * i + win_width * p[i][1]
          end = p[i][2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    se = events_curr
    if j == 0:
      start = 0.0
      end = start + win_length
      if preds.shape[0] > 1:
        end -= discard

      # print("start: {}   end: {}".format(start, end))
    elif j == n_preds - 1:
      start = j * hop_size + discard
      end = start - discard + win_length
      # print("start: {}   end: {}".format(start, end))

    else:
      start = j * hop_size + discard
      end = start + win_length - discard
      # print("start: {}   end: {}".format(start, end))

    for k in range(len(se)):
      se[k][0] = max(start, se[k][0] + j * hop_size)
      se[k][1] = min(end, se[k][1] + j * hop_size)

    # print(se)


    for see in se:
     events.append(see)

  # print(events)
  smooth_events = smoothe_events(events)

  return smooth_events

In [ ]:
mk_preds_YOHO("/content/development/TUT-sound-events-2017-development/audio-mono/street/a001.wav")

[[3.931, 16.254, 'car'],
 [22.13, 22.69, 'large vehicle'],
 [24.09, 28.02, 'large vehicle'],
 [27.456, 29.993, 'people walking'],
 [29.403, 33.057, 'large vehicle'],
 [31.374, 31.642, 'people walking'],
 [35.568, 36.976, 'large vehicle'],
 [41.161, 42.288, 'large vehicle'],
 [45.095, 45.924, 'car'],
 [47.042, 51.804, 'large vehicle'],
 [52.921, 53.201, 'large vehicle'],
 [54.881, 55.161, 'large vehicle'],
 [58.802, 68.052, 'large vehicle'],
 [70.575, 71.406, 'car'],
 [72.536, 72.808, 'car'],
 [75.055, 75.324, 'car'],
 [121.538, 129.929, 'car'],
 [131.335, 131.889, 'car'],
 [137.215, 138.042, 'car'],
 [139.172, 151.209, 'car'],
 [160.737, 165.485, 'car'],
 [166.612, 167.441, 'car'],
 [168.575, 173.321, 'car'],
 [174.456, 175.01, 'car']]

In [ ]:
mk_preds_CRNN(model, "/content/development/TUT-sound-events-2017-development/audio-mono/street/a001.wav")

[[0.0, 0.0, 'car'],
 [0.0, 4.51, 'people walking'],
 [1.96, 1.99, 'car'],
 [2.78, 29.99, 'car'],
 [25.32, 28.03, 'large vehicle'],
 [28.85, 36.06, 'large vehicle'],
 [30.33, 30.34, 'car'],
 [35.28, 49.59, 'car'],
 [37.0, 37.83, 'large vehicle'],
 [41.72, 69.39, 'large vehicle'],
 [50.96, 50.96, 'car'],
 [69.56, 77.38, 'car'],
 [78.4, 78.42, 'car'],
 [80.36, 80.4, 'car'],
 [80.85, 80.87, 'car'],
 [82.32, 82.34, 'car'],
 [85.04, 94.7, 'people speaking'],
 [86.24, 99.72, 'people walking'],
 [96.41, 96.63, 'people speaking'],
 [111.72, 111.75, 'people walking'],
 [113.68, 120.15, 'people walking'],
 [117.6, 133.69, 'car'],
 [122.11, 122.11, 'people walking'],
 [137.2, 152.88, 'car'],
 [155.85, 156.95, 'people walking'],
 [156.8, 156.8, 'car'],
 [160.16, 160.31, 'car'],
 [160.72, 178.95, 'car'],
 [178.24, 182.87, 'people walking']]

In [ ]:
a, win_ranges = construct_examples("/content/development/TUT-sound-events-2017-development/audio-mono/street/a001.wav", win_len=2.56,hop_len=1.96)

max_event_silence = 0.3
preds = np.zeros((len(a), 9, 18))
mss_in = np.zeros((len(a), 257, 40))

for i in range(len(a)):
  M = get_log_melspectrogram(a[i])
  mss_in[i, :, :] = M.T

preds = model.predict(mss_in)
events = []

for i in range(len(preds)):
  p = preds[i, :, :]
  events_curr = []
  # win_width = win_length / no_of_div

  events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

  events += events_curr

print(events)

class_set = set([c[2] for c in events])
class_wise_events = {}

for c in list(class_set):
  class_wise_events[c] = []


for c in events:
  class_wise_events[c[2]].append(c)


all_events = []

for k in list(class_wise_events.keys()):
  curr_events = class_wise_events[k]
  count = 0

  while count < len(curr_events) - 1:
    if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
      curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
      del curr_events[count + 1]
    else:
      count += 1

  all_events += curr_events

for i in range(len(all_events)):
  all_events[i][0] = round(all_events[i][0], 3)
  all_events[i][1] = round(all_events[i][1], 3)

all_events.sort(key=lambda x: x[0])

print(all_events)


[[5.42, 6.46, 'car'], [5.88, 6.09, 'car'], [6.85, 8.43, 'car'], [7.84, 10.39, 'car'], [9.8, 12.350000000000001, 'car'], [11.76, 14.3, 'car'], [13.74, 14.26, 'large vehicle'], [13.73, 16.26, 'large vehicle'], [13.73, 13.76, 'people walking'], [14.06, 15.020000000000001, 'people speaking'], [15.63, 15.950000000000001, 'car'], [15.69, 18.21, 'large vehicle'], [16.72, 16.78, 'people speaking'], [21.61, 21.73, 'large vehicle'], [21.779999999999998, 22.32, 'large vehicle'], [22.81, 23.169999999999998, 'large vehicle'], [23.81, 24.099999999999998, 'large vehicle'], [23.53, 26.07, 'large vehicle'], [25.33, 26.06, 'car'], [25.490000000000002, 28.03, 'large vehicle'], [27.450000000000003, 27.46, 'car'], [27.450000000000003, 29.330000000000002, 'large vehicle'], [27.540000000000003, 29.55, 'car'], [29.380000000000003, 29.53, 'large vehicle'], [29.6, 29.830000000000002, 'car'], [29.62, 29.86, 'large vehicle'], [29.91, 29.970000000000002, 'car'], [29.4, 29.88, 'car'], [29.419999999999998, 29.52, 'l

In [ ]:
preds_to_se((model.predict(mss_in) >= 0.5).astype(np.float), win_start=0.0)

ValueError: ignored

# Extract annotations

In [ ]:
import os.path

In [ ]:
def extract_labels(annotation_path):
  events = read_annotation(annotation_path)

  ann = [[float(e[2]), float(e[3]), e[4]] for e in events]

  n_label = "/content/eval-files/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [ ]:
os.mkdir("/content/eval-files/")

In [ ]:
for audio in val_files:
  extract_labels(audio.replace(".wav", ".ann").replace("audio-mono", "meta"))

In [ ]:
val_files

{'/content/development/TUT-sound-events-2017-development/audio-mono/street/a001.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/a012.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/a128.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b005.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b006.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b094.wav'}

In [ ]:
val_files

{'/content/development/TUT-sound-events-2017-development/audio-mono/street/a010.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/a013.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/a129.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b008.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b099.wav',
 '/content/development/TUT-sound-events-2017-development/audio-mono/street/b100.wav'}

In [ ]:
model.load_weights("/content/model-best-error-YamNet-fold4.h5")

In [ ]:
for audio in val_files:
  audio_file_path = audio
  see = mk_preds_YOHO(model, audio_file_path)
  n_label = n_label = "/content/eval-files/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.ann"

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

In [ ]:
destination = "/content/eval-files/"
test_set = glob.glob(destination + "*[0-9].ann")

print(test_set)

eval_path = "/content/"


file_list = [
    {
    'reference_file': tt,
    'estimated_file': tt.replace(".ann","-se-prediction.ann")
    }
    for tt in test_set
]

data = []

# Get used event labels
all_data = dcase_util.containers.MetaDataContainer()
for file_pair in file_list:
    reference_event_list = sed_eval.io.load_event_list(
        filename=file_pair['reference_file']
    )
    estimated_event_list = sed_eval.io.load_event_list(
        filename=file_pair['estimated_file']
    )

    data.append({'reference_event_list': reference_event_list,
                'estimated_event_list': estimated_event_list})

    all_data += reference_event_list

event_labels = all_data.unique_event_labels

# Start evaluating

# Create metrics classes, define parameters
segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=event_labels,
    time_resolution=1.0
)

event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
    event_label_list=event_labels,
    t_collar=1.0
)

# Go through files
for file_pair in data:
    segment_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

    event_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

# Get only certain metrics
overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

# Or print all metrics as reports

model_basename = "YamNet-fold4.h5"
seg_eval_basename = "seg eval " + model_basename.replace(".h5", "") + ".txt"
ev_eval_basename = "event eval " + model_basename.replace(".h5", "") + ".txt"
with open(os.path.join(eval_path, seg_eval_basename), mode='w') as fp:
  fp.write(str(segment_based_metrics))

with open(eval_path + "/seg eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(segment_based_metrics, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(eval_path, ev_eval_basename), mode = 'w') as fp:
  fp.write(str(event_based_metrics))

with open(eval_path + "/event eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(event_based_metrics, f, pickle.HIGHEST_PROTOCOL)

['/content/eval-files/a008.ann', '/content/eval-files/b007.ann', '/content/eval-files/b100.ann', '/content/eval-files/a013.ann', '/content/eval-files/b008.ann', '/content/eval-files/a127.ann', '/content/eval-files/b099.ann', '/content/eval-files/b003.ann', '/content/eval-files/a010.ann', '/content/eval-files/b095.ann', '/content/eval-files/b093.ann', '/content/eval-files/a124.ann', '/content/eval-files/b009.ann', '/content/eval-files/b006.ann', '/content/eval-files/a001.ann', '/content/eval-files/b091.ann', '/content/eval-files/a131.ann', '/content/eval-files/b005.ann', '/content/eval-files/a128.ann', '/content/eval-files/a012.ann', '/content/eval-files/a129.ann', '/content/eval-files/b098.ann', '/content/eval-files/a003.ann', '/content/eval-files/b094.ann']
Accuracy: 0.8629725883299301


In [ ]:
files = glob.glob("/content/SACRNN/*")

In [ ]:
files

['/content/SACRNN/event eval SACRNN-ensemble-tta-.txt',
 '/content/SACRNN/model-best-error-SACRNN-fold3.h5',
 '/content/SACRNN/model-best-error-SACRNN-fold2.h5',
 '/content/SACRNN/model-best-f1.h5',
 '/content/SACRNN/model-best-error-SACRNN-fold4.h5',
 '/content/SACRNN/seg eval SACRNN-ensemble-tta-.txt',
 '/content/SACRNN/model-best-error-SACRNN-fold1.h5']

In [ ]:
for f in files:
    shutil.move(f, '/content/SACRNN/')

In [ ]:
with ZipFile("/content/drive/MyDrive/TUT-sound-events-2017/Models/2-7-21/SACRNN.zip", 'w') as my_zip:
  for f in files:
    my_zip.write(f)

In [ ]:
overall_segment_based_metrics['f_measure']['f_measure']

0.35764375876577836

In [ ]:
!nvidia-smi

Sun Jun 27 19:04:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |   6941MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Test on final Evaluation set

In [ ]:
def mk_preds_YOHO_2(model, audio_path, no_of_div = 32, hop_size = 10.0, discard = 0.3, win_length = 10.0, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((1, 32, 30))
  mss_in = np.zeros((1, 1001, 40))

  for i in range(1):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)):
      for jjj in range(0, 10):
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges[i][0]
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
def mk_preds_CRNN_2(model, audio_path, no_of_div = 9, hop_size = 10.0, discard = 0.3, win_length = 10.0, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((len(a), 9, 18))
  mss_in = np.zeros((len(a), 1001, 40))

  for i in range(1):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div

    events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
def mk_ens_preds_CRNN(models, audio_path, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((len(a), 9, 18))
  mss_in = np.zeros((len(a), 257, 40))

  ensemble_preds = []

  yhats = []

  for i in range(len(a)):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  for model in models:
    # v = mss_in.shape[0]
    # tau = mss_in.shape[1]

    # X = mss_in.reshape((-1, 257, 40, 1))

    # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
    # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)

    # X = warped_frequency_time_sepctrogram

    yhats.append(model.predict(mss_in))

  yhats = np.array(yhats)
  # sum across ensembles
  preds = np.mean(yhats, axis=0)

  # preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div

    events_curr = preds_to_se(p, win_start = win_ranges[i][0], audio_clip_length=win_length)

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
def mk_ens_preds_YOHO(models, audio_path, no_of_div = 9, hop_size = 1.96, discard = 0.3, win_length = 2.56, max_event_silence = 0.3, sampling_rate = 44100):
  a, win_ranges = construct_examples(audio_path, win_len=win_length,hop_len=hop_size)

  preds = np.zeros((len(a), 9, 18))
  mss_in = np.zeros((len(a), 257, 40))

  for i in range(len(a)):
    M = get_log_melspectrogram(a[i])
    mss_in[i, :, :] = M.T

  ensemble_preds = []

  yhats = []

  # v = X.shape[0]
  # tau = X.shape[1]

  # warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
  # warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)

  # X = warped_frequency_time_sepctrogram

  for model in models:
    for i in range(10):
      v = mss_in.shape[0]
      tau = mss_in.shape[1]

      X = mss_in.reshape((-1, 257, 40, 1))

      warped_frequency_spectrogram = spec_augment_tensorflow.frequency_masking(X, v=v,  frequency_masking_para=8, frequency_mask_num=1)
      warped_frequency_time_sepctrogram = spec_augment_tensorflow.time_masking(warped_frequency_spectrogram, tau=tau, time_masking_para=25, time_mask_num=2)

      X = warped_frequency_time_sepctrogram

      yhats.append(model.predict(X))


  # yhats = [model.predict(mss_in) for model in models]
  yhats = np.array(yhats)
  # sum across ensembles
  preds = np.mean(yhats, axis=0)

  # preds = model.predict(mss_in)
  events = []

  for i in range(len(preds)):
    p = preds[i, :, :]
    events_curr = []
    win_width = win_length / no_of_div
    for j in range(len(p)):
      for jjj in range(0, 6):
        if p[j][jjj*3] >= 0.5:
          start = win_width * j + win_width * p[j][jjj*3+1] + win_ranges[i][0]
          end = p[j][jjj*3+2] * win_width + start
          events_curr.append([start, end, rev_class_dict[jjj]])

    events += events_curr


  class_set = set([c[2] for c in events])
  class_wise_events = {}

  for c in list(class_set):
    class_wise_events[c] = []


  for c in events:
    class_wise_events[c[2]].append(c)


  all_events = []

  for k in list(class_wise_events.keys()):
    curr_events = class_wise_events[k]
    count = 0

    while count < len(curr_events) - 1:
      if (curr_events[count][1] >= curr_events[count + 1][0]) or (curr_events[count + 1][0] - curr_events[count][1] <= max_event_silence):
        curr_events[count][1] = max(curr_events[count + 1][1], curr_events[count][1])
        del curr_events[count + 1]
      else:
        count += 1

    all_events += curr_events

  for i in range(len(all_events)):
    all_events[i][0] = round(all_events[i][0], 3)
    all_events[i][1] = round(all_events[i][1], 3)

  all_events.sort(key=lambda x: x[0])

  return all_events



In [ ]:
os.mkdir("/content/eval-files-final")

In [ ]:
audio_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/test/*.wav")

In [ ]:
test_files = glob.glob("/content/Urban-SED/URBAN-SED_v2.0.0/audio/test/*.wav")

In [ ]:
def extract_labels_3(annotation_path):
  events = read_annotation(annotation_path)

  ann = [[float(e[0]), float(e[1]), e[2]] for e in events]

  n_label = "/content/eval-files-final/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [ ]:
test_files[0]

'/content/Urban-SED/URBAN-SED_v2.0.0/audio/test/soundscape_test_uniform240.wav'

In [ ]:
for audio in test_files:
  extract_labels_3(audio.replace(".wav", ".txt").replace("audio", "annotations"))

In [ ]:
# os.mkdir("/content/drive/MyDrive/TUT-sound-events-2017/Models/2-7-21/")

In [ ]:
zip_name = "/content/drive/MyDrive/TUT-sound-events-2017/Models/2-7-21/SACRNN.zip"
with ZipFile(zip_name, 'r') as zip:
  zip.extractall()

In [ ]:
for i in range(4):
  models[i].load_weights("/content/content/SACRNN/model-best-error-SACRNN-fold" + str(i+1) + ".h5")

In [ ]:
for audio in test_files:
  audio_file_path = audio
  see = mk_preds_CRNN_2(model, audio_file_path)
  n_label = n_label = "/content/eval-files-final/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.txt"

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))


In [ ]:
destination = "/content/eval-files-final/"
test_set = glob.glob(destination + "*[0-9].txt")

print(test_set)

eval_path = "/content/"


file_list = [
    {
    'reference_file': tt,
    'estimated_file': tt.replace(".txt","-se-prediction.txt")
    }
    for tt in test_set
]

data = []

# Get used event labels
all_data = dcase_util.containers.MetaDataContainer()
for file_pair in file_list:
    reference_event_list = sed_eval.io.load_event_list(
        filename=file_pair['reference_file']
    )
    estimated_event_list = sed_eval.io.load_event_list(
        filename=file_pair['estimated_file']
    )

    data.append({'reference_event_list': reference_event_list,
                'estimated_event_list': estimated_event_list})

    all_data += reference_event_list

event_labels = all_data.unique_event_labels

# Start evaluating

# Create metrics classes, define parameters
segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=event_labels,
    time_resolution=1.0
)

event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
    event_label_list=event_labels,
    t_collar=1.0
)

# Go through files
for file_pair in data:
    segment_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

    event_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

# Get only certain metrics
overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

# Or print all metrics as reports

model_basename = "CRNN-error.h5"
seg_eval_basename = "seg eval " + model_basename.replace(".h5", "") + ".txt"
ev_eval_basename = "event eval " + model_basename.replace(".h5", "") + ".txt"
with open(os.path.join(eval_path, seg_eval_basename), mode='w') as fp:
  fp.write(str(segment_based_metrics))

with open(eval_path + "/seg eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(segment_based_metrics, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(eval_path, ev_eval_basename), mode = 'w') as fp:
  fp.write(str(event_based_metrics))

with open(eval_path + "/event eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(event_based_metrics, f, pickle.HIGHEST_PROTOCOL)

['/content/eval-files-final/soundscape_test_uniform451.txt', '/content/eval-files-final/soundscape_test_bimodal49.txt', '/content/eval-files-final/soundscape_test_bimodal62.txt', '/content/eval-files-final/soundscape_test_uniform388.txt', '/content/eval-files-final/soundscape_test_uniform463.txt', '/content/eval-files-final/soundscape_test_uniform256.txt', '/content/eval-files-final/soundscape_test_bimodal391.txt', '/content/eval-files-final/soundscape_test_unimodal583.txt', '/content/eval-files-final/soundscape_test_unimodal128.txt', '/content/eval-files-final/soundscape_test_uniform237.txt', '/content/eval-files-final/soundscape_test_unimodal523.txt', '/content/eval-files-final/soundscape_test_unimodal39.txt', '/content/eval-files-final/soundscape_test_unimodal146.txt', '/content/eval-files-final/soundscape_test_bimodal14.txt', '/content/eval-files-final/soundscape_test_bimodal470.txt', '/content/eval-files-final/soundscape_test_uniform115.txt', '/content/eval-files-final/soundscape_

In [ ]:
!cp "/content/seg eval CRNN.txt" "/content/drive/MyDrive/Urban-SED/Evaluation"

# On Evaluation

In [ ]:
def extract_labels(annotation_path):
  events = read_annotation(annotation_path)

  ann = [[float(e[2]), float(e[3]), e[4]] for e in events]

  n_label = "/content/eval-files-2/" + os.path.basename(annotation_path)

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in ann))

In [ ]:
# os.mkdir("/content/eval-files/")

In [ ]:
for audio in val_files:
  extract_labels(audio.replace(".wav", ".ann").replace("audio-mono", "meta"))

In [ ]:
for audio in val_files:
  audio_file_path = audio
  see = mk_preds_vector(audio_file_path)
  n_label = n_label = "/content/eval-files/" + os.path.basename(audio_file_path).replace(".wav" ,"") + "-se-prediction.ann"

  with open(n_label, 'w') as fp:
    fp.write('\n'.join('{},{},{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in see))

audio_clip_length_samples is 3858413
audio_clip_length_samples is 2932309
audio_clip_length_samples is 3441476
audio_clip_length_samples is 3217728
audio_clip_length_samples is 3562034
audio_clip_length_samples is 3849125


In [ ]:
# !pip install sed_eval

     |████████████████████████████████| 2.1MB 10.1MB/s 
  Created wheel for sed-eval: filename=sed_eval-0.2.1-cp37-none-any.whl size=26124 sha256=4a0d10640f3160d5c6254e62f22a16c677ba1c13432374d6c178805e71838d79
  Stored in directory: /root/.cache/pip/wheels/3c/72/5d/5fc941f98c583ce9d8adee7f13e24b46459aeded4125f9c369
  Created wheel for dcase-util: filename=dcase_util-0.2.18-cp37-none-any.whl size=2147233 sha256=e523506b66b038e8ad25f544ab32ce7d2aaf3e2f0c8025307e5fbd5748612a38
  Stored in directory: /root/.cache/pip/wheels/d1/5d/9e/c94cd8bfb80541c73d549c734cafcd3de994dbb117a6485c81
Successfully built sed-eval dcase-util


In [ ]:
import sed_eval
import dcase_util

In [ ]:
import pickle

In [ ]:
destination = "/content/eval-files/"
test_set = glob.glob(destination + "*[0-9].ann")

print(test_set)

eval_path = "/content/"


file_list = [
    {
    'reference_file': tt,
    'estimated_file': tt.replace(".ann","-se-prediction.ann")
    }
    for tt in test_set
]

data = []

# Get used event labels
all_data = dcase_util.containers.MetaDataContainer()
for file_pair in file_list:
    reference_event_list = sed_eval.io.load_event_list(
        filename=file_pair['reference_file']
    )
    estimated_event_list = sed_eval.io.load_event_list(
        filename=file_pair['estimated_file']
    )

    data.append({'reference_event_list': reference_event_list,
                'estimated_event_list': estimated_event_list})

    all_data += reference_event_list

event_labels = all_data.unique_event_labels

# Start evaluating

# Create metrics classes, define parameters
segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
    event_label_list=event_labels,
    time_resolution=1.0
)

event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
    event_label_list=event_labels,
    t_collar=1.0
)

# Go through files
for file_pair in data:
    segment_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

    event_based_metrics.evaluate(
        reference_event_list=file_pair['reference_event_list'],
        estimated_event_list=file_pair['estimated_event_list']
    )

# Get only certain metrics
overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

# Or print all metrics as reports

model_basename = "YamNet-4.h5"
seg_eval_basename = "seg eval " + model_basename.replace(".h5", "") + ".txt"
ev_eval_basename = "event eval " + model_basename.replace(".h5", "") + ".txt"
with open(os.path.join(eval_path, seg_eval_basename), mode='w') as fp:
  fp.write(str(segment_based_metrics))

with open(eval_path + "/seg eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(segment_based_metrics, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(eval_path, ev_eval_basename), mode = 'w') as fp:
  fp.write(str(event_based_metrics))

with open(eval_path + "/event eval " + model_basename.replace(".h5", "") + ".pickle", 'wb') as f:
  pickle.dump(event_based_metrics, f, pickle.HIGHEST_PROTOCOL)

['/content/eval-files/b095.ann', '/content/eval-files/a012.ann', '/content/eval-files/b005.ann', '/content/eval-files/a003.ann', '/content/eval-files/a128.ann', '/content/eval-files/a013.ann', '/content/eval-files/a008.ann', '/content/eval-files/b008.ann', '/content/eval-files/b094.ann', '/content/eval-files/b098.ann', '/content/eval-files/b100.ann', '/content/eval-files/b009.ann', '/content/eval-files/a131.ann', '/content/eval-files/b091.ann', '/content/eval-files/a001.ann', '/content/eval-files/b007.ann', '/content/eval-files/a129.ann', '/content/eval-files/b093.ann', '/content/eval-files/a010.ann', '/content/eval-files/a127.ann', '/content/eval-files/b006.ann', '/content/eval-files/a124.ann', '/content/eval-files/b099.ann', '/content/eval-files/b003.ann']
Accuracy: 0.8454506012148258
